In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import gradio as gr
import matplotlib.pyplot as plt

In [2]:
# Load dataset
df = pd.read_csv(r"C:\Users\jhari\Downloads\Economic Indicators And Inflation (1) (1).csv")

In [3]:
df

,Country,Year,GDP (in billion USD),Inflation Rate (%),Unemployment Rate (%),Economic Growth (%)
0,USA,2010,15000,1.64,9.63,2.55
1,USA,2011,15500,3.16,8.94,1.53
2,USA,2012,16000,2.07,8.10,2.28
3,USA,2013,16500,1.50,7.70,1.84
4,USA,2014,17000,1.62,7.25,2.53
...,...,...,...,...,...,...
299,Pakistan,2021,296,8.90,5.00,5.70
300,Pakistan,2022,350,12.30,5.00,6.00
301,Pakistan,2023,400,20.00,5.00,4.00
302,Pakistan,2024,450,25.00,5.00,3.00


In [4]:
# Clean column names
df.columns = df.columns.str.strip()

In [5]:
# Encode 'Country' column


label_encoder = LabelEncoder()
df['Country'] = label_encoder.fit_transform(df['Country'])

In [6]:
# Define features and targets
features = ['Country', 'Year']
target = ['GDP (in billion USD)', 'Inflation Rate (%)', 'Unemployment Rate (%)', 'Economic Growth (%)']

In [7]:
X = df[features]
y = df[target]

In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [10]:
model.fit(X_train, y_train[target])

RandomForestRegressor(random_state=42)

In [11]:
predictions = model.predict(X_test)

In [12]:
mse = mean_squared_error(y_test[target], predictions)
r2 = r2_score(y_test[target], predictions)

In [13]:
print("Overall Model Evaluation:")
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

Overall Model Evaluation:
Mean Squared Error: 9251.41678697598
R2 Score: 0.7439282296986255


In [14]:
# --- Function to evaluate model per indicator ---
def train_and_evaluate_random_forest(X_train, X_test, y_train, y_test):
    results = {}
    for i, col in enumerate(target):
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train[col])
        preds = model.predict(X_test)
        mse = mean_squared_error(y_test[col], preds)
        r2 = r2_score(y_test[col], preds)
        results[col] = {
            'MSE': round(mse, 3),
            'R2': round(r2, 3)
        }
    return results

In [15]:
# Evaluate Random Forest model
rf_results = train_and_evaluate_random_forest(X_train, X_test, y_train, y_test)

In [16]:
# Display results
print("Random Forest Results:")
print(pd.DataFrame(rf_results).T)


Random Forest Results:
                             MSE     R2
GDP (in billion USD)   38007.219  0.998
Inflation Rate (%)         3.982  0.680
Unemployment Rate (%)      0.357  0.932
Economic Growth (%)        7.116  0.573


In [17]:
def predict_and_visualize(country_name, year):
    try:
        country_encoded = label_encoder.transform([country_name])[0]
    except:
        return "Country not found in data.", None

    # Use DataFrame with column names
    input_features = pd.DataFrame([[country_encoded, int(year) + 1]], columns=['Country', 'Year'])
    
    prediction = model.predict(input_features)[0]
    indicators = target
    values = prediction

    plt.figure(figsize=(8, 5))
    bars = plt.bar(indicators, values, color=['blue', 'orange', 'green', 'red'])
    plt.title(f"Predicted Economic Indicators for {country_name} in {int(year)+1}")
    plt.ylabel("Value")
    plt.xticks(rotation=20)

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.2f}', va='bottom', ha='center')

    plt.tight_layout()
    plt.savefig("prediction_plot.png")
    plt.close()

    summary = (
        f"Predicted values for {country_name} in {int(year)+1}:\n"
        f"GDP: {values[0]:.2f} billion USD\n"
        f"Inflation Rate: {values[1]:.2f}%\n"
        f"Unemployment Rate: {values[2]:.2f}%\n"
        f"Economic Growth: {values[3]:.2f}%"
    )

    return summary, "prediction_plot.png"

In [18]:
# --- Gradio Interface ---
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 Economic Indicator Predictor with Visualization")
    gr.Markdown("Enter a country and year to predict next year's economic indicators with a visual bar chart.")

    with gr.Row():
        country_input = gr.Textbox(label="Country Name (e.g., India)")
        year_input = gr.Number(label="Current Year (e.g., 2024)", precision=0)

    predict_button = gr.Button("Predict")

    output_text = gr.Textbox(label="Prediction Summary")
    output_image = gr.Image(label="Indicator Visualization")

    predict_button.click(fn=predict_and_visualize,
                         inputs=[country_input, year_input],
                         outputs=[output_text, output_image])

# Launch app
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
